In [1]:
import os
import sys

from os.path import join, pardir
sys.path.append(pardir)

from bids import BIDSLayout
from itertools import product, chain

from nipype import config
from ica_wf import make_subject_ica_wf
#from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles

import numpy as np
from nipype.interfaces.fsl.maths import TemporalFilter
from nipype.interfaces.fsl.model import MELODIC
from nipype.interfaces.fsl.preprocess import SUSAN
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.pipeline.engine import Workflow, Node

220503-12:34:38,186 nipype.utils WARNING:
	 A newer version (1.7.1) of nipy/nipype is available. You are using 1.7.0


In [2]:
config.enable_debug_mode()

In [4]:
import numpy as np
from nipype.interfaces.fsl.maths import TemporalFilter
from nipype.interfaces.fsl.model import MELODIC
from nipype.interfaces.fsl.preprocess import SUSAN
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.pipeline.engine import Workflow, Node

def calc_susan_thresh(boldfile, maskfile, timeax=0, median_factor=.75):
    """
    Calculate the median value within brainmask and multiply with fixed
    factor to get an estimate of the contrast between background and brain
    for FSL's SUSAN.
    """
    from nilearn.masking import apply_mask
    import numpy as np
    data = apply_mask(boldfile, maskfile)
    med = np.median(data.mean(axis=timeax))
    del data  # suspect memory leak
    return med * median_factor

def separate_files(bold_mask_datapaths):
    print('---- BOLDMASKFILES: ', bold_mask_datapaths,'\n')
    print('---- bold_mask_datapaths[0]: ', bold_mask_datapaths[0],'\n')
    print('---- bold_mask_datapaths[1]: ', bold_mask_datapaths[1],'\n')
    boldfile = bold_mask_datapaths[0]
    maskfile = bold_mask_datapaths[1]
    return boldfile, maskfile

def make_subject_ica_wf():
    """
    Example Inputs:
        wf.inputs.inputspec.bold_file = '/../preproc_bold.nii.gz'
        wf.inputs.inputspec.mask_file = '/../brain_mask.nii.gz'
        wf.inputs.inputspec.tr = 1.5
        wf.inputs.inputspec.hpf = 120.0/tr
        wf.inputs.inputspec.fwhm = 4.0
        wf.inputs.inputspec.out_dir = '/results/melodic'
        wf.base_dir = '/..'
    
    Output:
        nipype workflow = combining precessing (smoothing/temporal filtering)
        and ICA for one functional run
    TODO: give TR as input (to this function) or infer from data?
    """
    # create input spec
    inputspec = Node(
        IdentityInterface(
            fields=['bold_mask_datapaths',
                    'tr',
                    'hpf',
                    'fwhm',
                    'out_dir']
                    ),
            name="inputspec")
    inputspec.iterables = [('bold_mask_datapaths', bold_mask_datapaths)]
    # create node to separate bold & mask file
    inputpaths = Node(
        Function(
            function=separate_files, input_names=['bold_mask_datapaths'], 
            output_names=['boldfile', 'maskfile']),
            name='inputpaths'
    )
    # create node for smoothing
    calcthresh = Node(
        Function(
            function=calc_susan_thresh, input_names=['boldfile', 'maskfile'], 
            output_names=['smooth_thresh']),
            name='calcthresh'
    )
    susan = Node(SUSAN(), name='susan') # requires 
    # ... temporal filtering
    tfilt = Node(TemporalFilter(), name='tfilt')
    # ... ICA
    melodic = Node(MELODIC(out_all=True, no_bet=True, report=True), name='melodic')
    
    # connect nodes in workflow
    wf = Workflow(name='melodicwf')
    wf.connect([
        (inputspec, inputpaths, [('bold_mask_datapaths', 'bold_mask_datapaths')]),
        (inputpaths, calcthresh, [('bold_file', 'boldfile'),
                                  ('mask_file', 'maskfile')]),
        (inputspec, susan, [('fwhm', 'fwhm')]),
        (inputpaths, susan, [('bold_file', 'in_file')]),
        (inputspec, tfilt, [('hpf', 'highpass_sigma')]),
        (inputspec, melodic, [('out_dir', 'out_dir'),
                              ('tr', 'tr_sec')]),
        (inputpaths, melodic, [('mask_file', 'mask')]),
        (calcthresh, susan, [('smooth_thresh', 'brightness_threshold')]),
        (susan, tfilt, [('smoothed_file', 'in_file')]),
        (tfilt, melodic, [('out_file', 'in_files')])
    ])
    return wf

In [ ]:
# Preproc & ICA Flow:
#
# (For every dataset:)
#   For every subject:
#     For every run:
#       For every task:
#         For every space:
#
#           1. extract filenames of bold & mask file
#          (2. feed into preproc.py)
#          (3. visualize output)
#          (return ICs)
#

In [ ]:
def get_datapaths(bids_layout, subject, session, run, task, space):
    """
    Extract mask and bold file paths for one subject for one task
    of one run for one type of space.
    
    Input: BIDSlayout, subject, ...  
    Output: bold_file, mask_file
    """
    # check if run and session are present
    run = None if run in ('0','00', '') else run
    session = None if session in ('0','00', '') else session
    # get paths
    bold_file = layout.get(
                        subject=subject,
                        run=run,
                        session=session,
                        task=task,
                        space=space,
                        extension='nii.gz',
                        suffix='bold',
                        return_type='filename'
                        )
    bold_file = [i for i in bold_file if "AROMA" not in i]
    
    mask_file = layout.get(
                        subject=subject,
                        run=run,
                        session=session,
                        task=task,
                        space=space,
                        extension='nii.gz',
                        suffix='mask',
                        return_type='filename'
                        )
    return bold_file[0], mask_file[0]

In [ ]:
def read_paths(bids_layout, subject="all", session="all", run="all", task="all", space="all"):
    """
    Check if all data paths or only specific paths are asked for and give them back.
    
    Input: BIDSlayout
    Optional Input: subject, session, run, task, space
    Output: one file with all bold and mask file paths as tuples
    """
    if all([param == "all" for param in (subject, session, run, task, space)]):
        subject = bids_layout.get(return_type='id', target='subject', desc='preproc')
        session = bids_layout.get(return_type='id', target='session', desc='preproc')
        run = bids_layout.get(return_type='id', target='session', desc='preproc')
        task = bids_layout.get(return_type='id', target='task', desc='preproc')
        space = bids_layout.get(return_type='id', target='space', desc='preproc')
    else:
        subject = subject
        session = [session] # TODO: for many runs/session, does pybids just give a nr or a list with the sessions?
        run = [run]
        task = [task]
        space = [space]
    
    # check if run and session are present
    session = '0' if session == [] else session
    run = '0' if run == [] else run
    
    # all combinations
    combinations = list(product(subject, session, run, task, space))
    print(combinations)
    #bold_files, mask_files = zip(*[get_datapaths(bids_layout, *params) for params in combinations])
    bold_mask_files = [get_datapaths(bids_layout, *params) for params in combinations]
    
    return bold_mask_files #bold_files, mask_files 

In [ ]:
# create layout from BIDS dataset
bidsdata_dir = '/LOCAL/jzerbe/faces_vs_houses/ds002938'
layout = BIDSLayout(bidsdata_dir, derivatives=True)

In [ ]:
# testing: get all data paths
datapaths = read_paths(layout)
datapaths

In [ ]:
# testing: get data paths for one subject for certain parameters
subjects = ['03']
sessions = '0' # per subject
runs = '0'
tasks = 'effort'
spaces = 'T1w'

bold_mask_datapaths = read_paths(layout, subjects, sessions, runs, tasks, spaces)
#list_bold = next(zip(*datapaths_twosub))
#list_bold
##datapaths_twosub
#list_bold, list_mask = read_paths(layout, subjects, sessions, runs, tasks, spaces)
#list_bold = list(list_bold)
#list_mask = list(list_mask)
#list_bold
bold_mask_datapaths

In [ ]:
## TESTING: combine read_paths() with make_subject_ica_wf()

# path parameters
subjects = ['03', '08']
sessions = '0' # per subject
runs = '0'
tasks = 'effort'
spaces = 'T1w'

# workflow parameters
p_tr = 1.5
p_hpf_sec =120.0
p_hpf = float(p_hpf_sec/p_tr)
p_fwhm = 4.0
base_dir = '/LOCAL/jzerbe/temp_results'
out_dir = '/LOCAL/jzerbe/temp_results/melodic/sub-03_task-effort_space-T1w_melodic' # TODO flexible folder name

# make workflow
melodicwf = make_subject_ica_wf()

# get paths
#list_bold, list_mask = read_paths(layout, subjects, sessions, runs, tasks, spaces)
#bold_mask_files = read_paths(layout, subjects, sessions, runs, tasks, spaces)
bold_mask_datapaths = read_paths(layout, subjects, sessions, runs, tasks, spaces)
bold_mask_datapaths

In [ ]:
# put params & paths in workflow
melodicwf.inputs.inputspec.hpf = p_hpf
melodicwf.inputs.inputspec.tr = p_tr
melodicwf.inputs.inputspec.fwhm = p_fwhm
melodicwf.inputs.inputspec.out_dir = out_dir
melodicwf.inputs.inputspec.bold_mask_datapaths = bold_mask_datapaths
#melodicwf.inputs.inputspec.iterables = [('bold_mask_datapaths', bold_mask_datapaths)]
melodicwf.base_dir = base_dir

# TODO: include this in preproc: ()maybe this makes it go through the first bold + all masks, 2nd bold + all masks, ... etc
#inputspec = Node(IdentityInterface(fields=['boldfile', 'maskfile']),
#                  name="inputspec")
#inputspec.iterables = [('boldfile', boldfile),
#                        ('maskfile', maskfile)]

#inputspec = Node(
#        IdentityInterface(
#            fields=['bold_mask_datapaths',
#                    'bold_file',
#                    'mask_file',
#                    'tr',
#                    'hpf',
#                    'fwhm',
#                    'out_dir']
#                    ),
#            name="inputspec")

# define this outside of workflow when calling it:
#inputspec.iterables = [('bold_mask_datapaths', bold_mask_datapaths)]
#
## do this again inside:
#datapaths = Node(IdentityInterface(fields=['bold_file','mask_file']),name="datapaths")
#
#def separate_files(bold_mask_datapaths):
#    boldfile = bold_mask_datapaths[0]
#    maskfile = bold_mask_datapaths[1]
#    return boldfile, maskfile

## create node to read out bold and mask files
#    separatefiles = Node(
#        Function(
#            function=separate_files, input_names=['bold_mask_datapaths'], 
#            output_names=['boldfile', 'maskfile']),
#            name='calcthresh'
#    )
#

#templates = {'bold': bold_mask_datapaths[0],
#             'mask': bold_mask_datapaths[1]}
#selectfiles = Node(SelectFiles(templates), #,base_directory='/data/ds000114'
#                   name="selectfiles")
#
#for bold_file, mask_file in datapaths_twosub: # [(x,y),(x,y)]
#    melodicwf.inputs.inputspec.bold_file = list(list_bold)
#    melodicwf.inputs.inputspec.mask_file = list(list_mask)
#    # run workflow
#    melodicwf.run()
    
# output: melodic directory with files

In [ ]:
boldd, maskk = separate_files(bold_mask_datapaths)

In [ ]:
#print('Original datapaths: ', bold_mask_datapaths, '\n')
#first_elem = bold_mask_datapaths[0]
#print('First: ', first_elem, '\n')
#ff_elem = first_elem[0]
#print('FF: ', ff_elem)
#SelectFiles?
melodicwf.run()

In [ ]:
inputspecs = Node(IdentityInterface(fields=['subject_id', 'task_name', 'space']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list),
                        ('task_name', task_list),
                        ('space')]

# SelectFiles - to grab the data (alternativ to DataGrabber)
bold_file = layout.get(
                        subject=subject,
                        run=run,
                        session=session,
                        task=task,
                        space=space,
                        extension='nii.gz',
                        suffix='bold',
                        return_type='filename'
                        )
#opj('derivatives', 'fmriprep', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_t1w_preproc.nii.gz')
mask_file = opj('sub-{subject_id}', 'ses-test', 'func',
                'sub-{subject_id}_ses-test_task-{task_name}_bold.nii.gz')

templates = {'bold': bold_file,
             'mask': mask_file}
selectfiles = Node(SelectFiles(templates,
                               base_directory='/data/ds000114'),
                   name="selectfiles")

In [ ]:
tr = 1.5
hpf = 100.
fwhm = 4.
bold_files = ... # list of strings
mask_files = ... # list of strings
out_dirs = ... # list of strings

In [ ]:
runwfs = []
for boldfile, maskfile, outdir in zip(bold_files, mask_files, out_dirs):
    runwf = make_run_ica_wf()
    runwf.inputspec.inputs.bold_file = boldfile
    runwf.inputspec.inputs.mask_file = mask_file
    runwf.inputspec.inputs.out_dir = outdir
    runwfs.append(runwf)

dataset_wf = Workflow(name='dataset_wf')
dataset_wf.add_nodes(runwfs)
dataset_wf.run(plugin='MultiProc', plugin_args={nproc=10})